In [ ]:
%pip install opencv-python torch torchvision  # you may also need to install other dependencies

In [3]:
from huggingface_hub import login, snapshot_download
import os

login(token="", add_to_git_credential=True)
model_names = [
        "Cosmos-0.1-Tokenizer-CI16x16",
]
for model_name in model_names:
    hf_repo = "nvidia/" + model_name
    local_dir = "../pretrained_ckpts/" + model_name
    os.makedirs(local_dir, exist_ok=True)
    print(f"downloading {model_name}...")
    snapshot_download(repo_id=hf_repo, local_dir=local_dir)

/home/ali/PycharmProjects/Cosmos-Tokenizer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
downloading Cosmos-0.1-Tokenizer-CI16x16...


Fetching 7 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.52s/it]


In [4]:
# If you haven't installed the Cosmos Tokenizer yet, do so first.
# Also ensure you have a GPU runtime available, as the tokenizer requires CUDA.
# If you cloned the repository:
# %pip install -e /path/to/Cosmos-Tokenizer

import cv2
import numpy as np
import torch

import importlib
import cosmos_tokenizer.image_lib

importlib.reload(cosmos_tokenizer.image_lib)
from cosmos_tokenizer.image_lib import ImageTokenizer

# 1) Specify the model name, and the paths to the encoder/decoder checkpoints.
model_name = "Cosmos-0.1-Tokenizer-CI16x16"
encoder_ckpt = f"../pretrained_ckpts/{model_name}/encoder.jit"
decoder_ckpt = f"../pretrained_ckpts/{model_name}/decoder.jit"

# 2) Load or provide the image filename you want to tokenize & reconstruct.
input_image_path = "../test_data/image.png"

# 3) Read the image from disk (shape = H x W x 3 in BGR). Then convert to RGB.
original_bgr = cv2.imread(input_image_path)
if original_bgr is None:
    raise FileNotFoundError(f"Could not read image file: {input_image_path}")

original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)

# 4) Expand dimensions to B x H x W x C, since the ImageTokenizer expects a batch dimension
#    in the input. (Batch size = 1 in this example.)
input_image = np.expand_dims(original_rgb, axis=0)

# 5) Create the ImageTokenizer instance with the encoder & decoder.
#    - device="cuda" uses the GPU
#    - dtype="bfloat16" expects Ampere or newer GPU (A100, RTX 30xx, etc.)
tokenizer = ImageTokenizer(
    checkpoint_enc=encoder_ckpt,
    checkpoint_dec=decoder_ckpt,
    device="cpu",
    dtype="bfloat16",
)

# 6) Use the tokenizer to autoencode (encode & decode) the image.
#    The output is a NumPy array with shape = B x H x W x C, range [0..255].
reconstructed_image = tokenizer.forward(input_image)

# 7) Extract the single image from the batch (index 0), convert to uint8.
reconstructed_image = reconstructed_image[0].astype(np.uint8)

# 8) Convert from RGB back to BGR (if you want to save using OpenCV).
reconstructed_bgr = cv2.cvtColor(reconstructed_image, cv2.COLOR_RGB2BGR)

# 9) Save the reconstructed image to disk.
output_image_path = "my_image_reconstructed.jpg"
cv2.imwrite(output_image_path, reconstructed_bgr)

print("Reconstruction saved to:", output_image_path)


Reconstruction saved to: my_image_reconstructed.jpg
